In [ ]:
from litellm import completion
import os

In [ ]:

import os
import base64
from openai import AzureOpenAI

subscription_key = "CpcP5WHUW180iluo6CudtpwSnjjvwcEqNbpq8cdVsXu3zkMQqQVgJQQJ99BBACHYHv6XJ3w3AAAAACOGPcpj"

endpoint = os.getenv("ENDPOINT_URL", "https://ai-21t10208850765ai125704117754.cognitiveservices.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4.1")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", subscription_key)

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)


def completion_func(message : str = "") : 
    response = client.chat.completions.create(
        model=deployment,
        messages=message,
        max_tokens=4090,
        temperature=0.0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        # stop=None,
        # stream=False
    )
    content = response.choices[0].message.content
    if content is None:
        # Kiểm tra kết quả bộ lọc nội dung để xác định nguyên nhân
        filter_results = response.choices[0].message.content_filter_results
    if filter_results and filter_results.get('violence', {}).get('filtered', False):
        print("Nội dung bị chặn do có yếu tố bạo lực.")
    else:
        print("Không có nội dung nào được trả về.")
    
    # Xử lý nội dung bình thường
    print("Nội dung trả về: ", content)
    return response


In [8]:
prompt = """
Bạn là một hệ thống sinh test case tự động cho các bài toán lập trình.
Hãy sinh ra K test case đa dạng (bao gồm cả edge case) cho bài toán sau, mỗi test case gồm input và output tương ứng.
Nếu có thể, hãy giải thích ngắn gọn ý nghĩa của từng test case.

Ví dụ:
Bài_toán: Tính tổng hai số
Mô_tả: Viết một hàm nhận vào hai số và trả về tổng của chúng. Input là a và b, output là tổng của a và b. Giới hạn của a và b là từ 0 đến 10^9.

Kết quả mong muốn (dưới dạng JSON):
[
    {{"input": [3, 5], "output": 8, "description": "Tổng hai số nguyên dương."}},
    {{"input": [0, 0], "output": 0, "description": "Cả hai số là 0."}},
    {{"input": [-1000, 20000], "output": 19000, "description": "Một số âm, một số dương."}},
    {{"input": [1000000000, 1000000000], "output": 2000000000, "description": "Cả hai số đạt giá trị lớn nhất."}},
    {{"input": [1, 999999999], "output": 1000000000, "description": "Một số nhỏ nhất, một số lớn nhất."}},
    {{"input": [-500, -500], "output": -1000, "description": "Cả hai số đều âm."}}
]

Bài toán: {problem}
Hãy sinh {K} test case theo định dạng trên.
Chỉ trả về kết quả cuối cùng dưới dạng JSON, không kèm bất kỳ giải thích hoặc văn bản nào khác.
"""

In [10]:
from neo4j_graphrag.llm import AzureOpenAILLM
from dotenv import load_dotenv
import os
import json
dotenv_path = "D:/UbuntuSystem/OnlineCodingCompetition/RecommendationSystem/.env"
load_dotenv(dotenv_path)

subscription_key = os.getenv("OPENAI_KEY")
endpoint = os.getenv("endpoint")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME", "gpt-4.1")
api_version= os.getenv("api_version", "2025-01-01-preview")
if not subscription_key or not endpoint:
    raise ValueError("Please set the OPENAI_KEY and endpoint environment variables.")

llm = AzureOpenAILLM(
    model_name=DEPLOYMENT_NAME,
    azure_endpoint=endpoint,
    api_version=api_version, 
    api_key=subscription_key,
    # temperature=0.7,
    # max_tokens=4096,
    # top_p=0.6,
)


def generate_test_cases(code: str, K_candidates: int, problem_id : int) -> [dict]:
    """
    Generate test cases for a given code snippet based on the provided prompt.
    
    Args:
        code (str): The code snippet for which to generate test cases.
        K (int): The number of initial test cases to generate.
        N (int): The number of additional test cases to generate.
    
    Returns:
        str: A formatted string containing the generated test cases.
    """
    # Format the prompt with the specified number of test cases
    formatted_prompt = prompt.format(K=K_candidates, problem=code)
    response = llm.invoke(formatted_prompt)
    if not response or not response.content:
        raise ValueError("No response received from the LLM.")
    response_content = response.content.strip()
    result = json.loads(response_content)
    return result
title = "Nhân 2 số nguyên"
description = "Viết một hàm nhận vào hai số và trả về tích của chúng. Input là a và b, output là tích của a và b. Giới hạn của a và b là từ 0 đến 10^9."
response = generate_test_cases(f"Bài_toán:{title}/n Mô_tả:{description}",5,6)


In [11]:
print(type(response))

<class 'list'>
